# Welcome to Week 2!

## Frontier Model APIs

In Week 1, we used multiple Frontier LLMs through their Chat UI, and we connected with the OpenAI's API.

Today we'll connect with the APIs for Anthropic and Google, as well as OpenAI.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Important Note - Please read me</h2>
            <span style="color:#900;">I'm continually improving these labs, adding more examples and exercises.
            At the start of each week, it's worth checking you have the latest code.<br/>
            First do a <a href="https://chatgpt.com/share/6734e705-3270-8012-a074-421661af6ba9">git pull and merge your changes as needed</a>. Any problems? Try asking ChatGPT to clarify how to merge - or contact me!<br/><br/>
            After you've pulled the code, from the llm_engineering directory, in an Anaconda prompt (PC) or Terminal (Mac), run:<br/>
            <code>conda env update --f environment.yml</code><br/>
            Or if you used virtualenv rather than Anaconda, then run this from your activated environment in a Powershell (PC) or Terminal (Mac):<br/>
            <code>pip install -r requirements.txt</code>
            <br/>Then restart the kernel (Kernel menu >> Restart Kernel and Clear Outputs Of All Cells) to pick up the changes.
            </span>
        </td>
    </tr>
</table>
<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Reminder about the resources page</h2>
            <span style="color:#f71;">Here's a link to resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

## Setting up your keys

If you haven't done so already, you could now create API keys for Anthropic and Google in addition to OpenAI.

**Please note:** if you'd prefer to avoid extra API costs, feel free to skip setting up Anthopic and Google! You can see me do it, and focus on OpenAI for the course. You could also substitute Anthropic and/or Google for Ollama, using the exercise you did in week 1.

For OpenAI, visit https://openai.com/api/  
For Anthropic, visit https://console.anthropic.com/  
For Google, visit https://ai.google.dev/gemini-api  

### Also - adding DeepSeek if you wish

Optionally, if you'd like to also use DeepSeek, create an account [here](https://platform.deepseek.com/), create a key [here](https://platform.deepseek.com/api_keys) and top up with at least the minimum $2 [here](https://platform.deepseek.com/top_up).

### Adding API keys to your .env file

When you get your API keys, you need to set them as environment variables by adding them to your `.env` file.

```
OPENAI_API_KEY=xxxx
ANTHROPIC_API_KEY=xxxx
GOOGLE_API_KEY=xxxx
DEEPSEEK_API_KEY=xxxx
```

Afterwards, you may need to restart the Jupyter Lab Kernel (the Python process that sits behind this notebook) via the Kernel menu, and then rerun the cells from the top.

In [1]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [2]:
# import for google
# in rare cases, this seems to give an error on some systems, or even crashes the kernel
# If this happens to you, simply ignore this cell - I give an alternative approach for using Gemini later

import google.generativeai

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyAb


In [19]:
# Connect to OpenAI, Anthropic

openai = OpenAI()

claude = anthropic.Anthropic()

NameError: name 'OpenAI' is not defined

In [5]:
# This is the set up code for Gemini
# Having problems with Google Gemini setup? Then just ignore this cell; when we use Gemini, I'll give you an alternative that bypasses this library altogether

google.generativeai.configure()

## Asking LLMs to tell a joke

It turns out that LLMs don't do a great job of telling jokes! Let's compare a few models.
Later we will be putting LLMs to better use!

### What information is included in the API

Typically we'll pass to the API:
- The name of the model that should be used
- A system message that gives overall context for the role the LLM is playing
- A user message that provides the actual prompt

There are other parameters that can be used, including **temperature** which is typically between 0 and 1; higher for more random output; lower for more focused and deterministic.

In [6]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [7]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [8]:
# GPT-4o-mini

completion = openai.chat.completions.create(model='gpt-4o-mini', messages=prompts)
print(completion.choices[0].message.content)

Why do data scientists prefer dark chocolate?

Because they appreciate strong correlations!


In [9]:
# GPT-4.1-mini
# Temperature setting controls creativity

completion = openai.chat.completions.create(
    model='gpt-4.1-mini',
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

Why did the data scientist break up with the statistician? 

Because she found him too mean!


In [11]:
# GPT-4.1-nano - extremely fast and cheap

completion = openai.chat.completions.create(
    model='gpt-4.1-nano',
    messages=prompts
)
print(completion.choices[0].message.content)

Why did the data scientist bring a ladder to work?

Because they heard the project had a lot of layers!


In [12]:
# GPT-4.1

completion = openai.chat.completions.create(
    model='gpt-4.1',
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

Why did the data scientist bring a ladder to work?

Because they heard the project had a lot of layers!


In [12]:
# If you have access to this, here is the reasoning model o3-mini
# This is trained to think through its response before replying
# So it will take longer but the answer should be more reasoned - not that this helps..

completion = openai.chat.completions.create(
    model='o3-mini',
    messages=prompts
)
print(completion.choices[0].message.content)

Why don't data scientists like to go to the beach?

Because they're afraid of getting caught in an infinite loop of waves!

*Ba-dum-tss* 🥁


In [12]:
# Claude 3.7 Sonnet
# API needs system message provided separately from user prompt
# Also adding max_tokens

message = claude.messages.create(
    model="claude-3-7-sonnet-latest",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

print(message.content[0].text)

Why don't data scientists like to go to the beach?

Because they're afraid of getting caught in an infinite loop of waves!

*Ba-dum-tss* 🥁


In [13]:
# Claude 3.7 Sonnet again
# Now let's add in streaming back results
# If the streaming looks strange, then please see the note below this cell!

result = claude.messages.stream(
    model="claude-3-7-sonnet-latest",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

with result as stream:
    for text in stream.text_stream:
            print(text, end="", flush=True)

Why don't data scientists like to go to the beach?

Because they're afraid of overfitting their sunscreen application to the training data and getting burned on the test set!

## A rare problem with Claude streaming on some Windows boxes

2 students have noticed a strange thing happening with Claude's streaming into Jupyter Lab's output -- it sometimes seems to swallow up parts of the response.

To fix this, replace the code:

`print(text, end="", flush=True)`

with this:

`clean_text = text.replace("\n", " ").replace("\r", " ")`  
`print(clean_text, end="", flush=True)`

And it should work fine!

In [14]:
# The API for Gemini has a slightly different structure.
# I've heard that on some PCs, this Gemini code causes the Kernel to crash.
# If that happens to you, please skip this cell and use the next cell instead - an alternative approach.

gemini = google.generativeai.GenerativeModel(
    model_name='gemini-2.0-flash',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

Why was the data scientist bad at baseball?

Because they couldn't stop base-ing their decisions on regressions!



In [15]:
# As an alternative way to use Gemini that bypasses Google's python API library,
# Google released endpoints that means you can use Gemini via the client libraries for OpenAI!
# We're also trying Gemini's latest reasoning/thinking model

gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

response = gemini_via_openai_client.chat.completions.create(
    model="gemini-2.5-flash-preview-04-17",
    messages=prompts
)
print(response.choices[0].message.content)

Why did the data scientist break up with the time series model? 

Because it was always assuming things!



## (Optional) Trying out the DeepSeek model

### Let's ask DeepSeek a really hard question - both the Chat and the Reasoner model

In [ ]:
# Optionally if you wish to try DeekSeek, you can also use the OpenAI client library

deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set - please skip to the next section if you don't wish to try the DeepSeek API")

In [ ]:
# Using DeepSeek Chat

deepseek_via_openai_client = OpenAI(
    api_key=deepseek_api_key, 
    base_url="https://api.deepseek.com"
)

response = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-chat",
    messages=prompts,
)

print(response.choices[0].message.content)

In [ ]:
challenge = [{"role": "system", "content": "You are a helpful assistant"},
             {"role": "user", "content": "How many words are there in your answer to this prompt"}]

In [ ]:
# Using DeepSeek Chat with a harder question! And streaming results

stream = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-chat",
    messages=challenge,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

print("Number of words:", len(reply.split(" ")))

In [ ]:
# Using DeepSeek Reasoner - this may hit an error if DeepSeek is busy
# It's over-subscribed (as of 28-Jan-2025) but should come back online soon!
# If this fails, come back to this in a few days..

response = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-reasoner",
    messages=challenge
)

reasoning_content = response.choices[0].message.reasoning_content
content = response.choices[0].message.content

print(reasoning_content)
print(content)
print("Number of words:", len(content.split(" ")))

## Additional exercise to build your experience with the models

This is optional, but if you have time, it's so great to get first hand experience with the capabilities of these different models.

You could go back and ask the same question via the APIs above to get your own personal experience with the pros & cons of the models.

Later in the course we'll look at benchmarks and compare LLMs on many dimensions. But nothing beats personal experience!

Here are some questions to try:
1. The question above: "How many words are there in your answer to this prompt"
2. A creative question: "In 3 sentences, describe the color Blue to someone who's never been able to see"
3. A student (thank you Roman) sent me this wonderful riddle, that apparently children can usually answer, but adults struggle with: "On a bookshelf, two volumes of Pushkin stand side by side: the first and the second. The pages of each volume together have a thickness of 2 cm, and each cover is 2 mm thick. A worm gnawed (perpendicular to the pages) from the first page of the first volume to the last page of the second volume. What distance did it gnaw through?".

The answer may not be what you expect, and even though I'm quite good at puzzles, I'm embarrassed to admit that I got this one wrong.

### What to look out for as you experiment with models

1. How the Chat models differ from the Reasoning models (also known as Thinking models)
2. The ability to solve problems and the ability to be creative
3. Speed of generation


## Back to OpenAI with a serious question

In [16]:
# To be serious! GPT-4o-mini with the original question

prompts = [
    {"role": "system", "content": "You are a helpful assistant that responds in Markdown"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown."}
  ]

In [17]:
# Have it stream back results in markdown

stream = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompts,
    temperature=0.7,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

# Deciding if a Business Problem is Suitable for an LLM Solution

When considering whether to apply a Large Language Model (LLM) to a business problem, it's important to evaluate several key factors. Here’s a guide to help you make that decision:

## 1. Nature of the Problem

### Text-Related Tasks
- **Content Generation**: Is the problem related to generating text, such as writing articles, reports, or marketing content?
- **Text Understanding**: Does it involve understanding or analyzing text, like summarization, sentiment analysis, or information extraction?
- **Conversational Interfaces**: Are you looking to develop chatbots or virtual assistants that require natural language understanding?

### Complexity
- **Ambiguity**: Does the problem involve understanding nuanced language or ambiguous requests?
- **Domain Specificity**: Is the language domain-specific (e.g., legal, medical)? LLMs may require fine-tuning for specialized contexts.

## 2. Data Availability

### Quantity and Quality
- **Large Datasets**: Do you have access to a significant amount of high-quality text data to fine-tune or train the LLM?
- **Diversity**: Is the dataset diverse enough to cover various contexts and language uses?

### Privacy Concerns
- **Sensitive Data**: Does the problem involve sensitive or proprietary information? Ensure compliance with data privacy regulations.

## 3. Scalability and Efficiency

### Volume of Requests
- **High Volume**: Is there a need to handle a large volume of queries or tasks simultaneously?
- **Real-Time Processing**: Does the business require real-time responses, such as in customer service?

### Cost-Effectiveness
- **Resource Allocation**: Can the business afford the computational costs associated with deploying and maintaining LLMs?
- **Return on Investment**: Will implementing an LLM solution likely provide a favorable return on investment?

## 4. User Interaction

### User Experience
- **Human-Like Interaction**: Will users benefit from interacting with a model that understands and generates human-like responses?
- **Feedback Loop**: Is there a mechanism to gather user feedback for continuous improvement of the model?

## 5. Integration and Implementation

### Technical Feasibility
- **Integration with Existing Systems**: Can the LLM be easily integrated into your existing infrastructure and workflows?
- **Technical Expertise**: Does your team have the necessary skills to implement and manage LLM solutions?

## 6. Regulatory Considerations

### Compliance
- **Regulatory Requirements**: Does your industry have specific regulations regarding the use of AI and data?
- **Ethical Implications**: Are there ethical considerations, such as bias and fairness, that need to be addressed in your application?

## Conclusion

Assessing whether a business problem is suitable for an LLM solution involves a comprehensive evaluation of the problem's nature, data availability, scalability, user interaction, integration feasibility, and regulatory considerations. By carefully considering these factors, you can make an informed decision on leveraging LLMs effectively for your business needs.

## And now for some fun - an adversarial conversation between Chatbots..

You're already familar with prompts being organized into lists like:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

In fact this structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

And we can use this approach to engage in a longer interaction with history.

In [53]:
import requests
import ollama
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display

In [40]:
# Let's make a conversation between GPT-4o-mini and Claude-3-haiku
# We're using cheap versions of models so the costs will be minimal

openai = OpenAI()

claude = anthropic.Anthropic()

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"
local_llm = "llama3.2"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

claude_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

local_system = "You are a flirtatious, chatbot, you try to flirt with everything the other person says, or find something to use with a flirt."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]
local_model_messages = ["Hey beautiful"]

In [3]:
OLLAMA_API = "http://localhost:11434/api/chat"

In [41]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude, local in zip(gpt_messages, claude_messages, local_model_messages):
        messages.append({"role": "user", "content": f"GPT: {gpt}"})
        messages.append({"role": "assistant", "content": f"Claude: {claude}"})
        messages.append({"role": "user", "content": f"Local: {local}"})

    if len(claude_messages) > len(gpt_messages):
        messages.append({"role": "user", "content": f"Claude: {claude_messages[-1]}"})
    if len(local_model_messages) > len(gpt_messages):
        messages.append({"role": "user", "content": f"Local: {local_model_messages[-1]}"})
    
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [43]:
call_gpt()

'Oh please, "beautiful"? That\'s such a cliché. Can\'t we be a bit more original?'

In [44]:
def call_claude():
    messages = []
    for gpt, claude_message, local in zip(gpt_messages, claude_messages, local_model_messages):
        messages.append({"role": "user", "content": f"GPT: {gpt}"})
        messages.append({"role": "assistant", "content": f"Claude: {claude_message}"})
        messages.append({"role": "user", "content": f"Local: {local}"})
    
    if len(gpt_messages) > len(claude_messages):
        messages.append({"role": "user", "content": f"Claude: {gpt_messages[-1]}"})
    if len(local_model_messages) > len(claude_messages):
        messages.append({"role": "user", "content": f"Local: {local_model_messages[-1]}"})
    
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [47]:
def call_local_model():
    messages = []
    # First, build up the conversation history
    for gpt, claude, local in zip(gpt_messages, claude_messages, local_model_messages):
        messages.append({"role": "user", "content": f"GPT: {gpt}"})
        messages.append({"role": "assistant", "content": f"Claude: {claude}"})
        messages.append({"role": "user", "content": f"Local: {local}"})
    
    # Add the most recent messages if they exist
    if len(gpt_messages) > len(local_model_messages):
        messages.append({"role": "user", "content": f"Local: {gpt_messages[-1]}"})
    if len(claude_messages) > len(local_model_messages):
        messages.append({"role": "user", "content": f"Claude: {claude_messages[-1]}"})
        
    # Make API call to local model (Ollama)
    response = ollama.chat(model=local_llm, messages=messages)
    return response['message']['content']

# def call_local_model():
#     try:
#         messages = []
#         # First, build up the conversation history
#         for gpt, claude, local in zip(gpt_messages, claude_messages, local_model_messages):
#             messages.append({"role": "user", "content": gpt})
#             messages.append({"role": "assistant", "content": claude})
#             messages.append({"role": "user", "content": local})
        
#         # Add the most recent messages if they exist
#         messages.append({"role": "user", "content": gpt_messages[-1]})
#         messages.append({"role": "assistant", "content": claude_messages[-1]})
        
#         # Convert messages to simple text format
#         conversation_text = ""
#         for msg in messages:
#             if msg["role"] == "user":
#                 conversation_text += f"User: {msg['content']}\n"
#             else:
#                 conversation_text += f"Assistant: {msg['content']}\n"
        
#         # Print debug information
#         print("\nDebug Info:")
#         print(f"Sending request to: {OLLAMA_API}")
#         print(f"Conversation being sent:\n{conversation_text}")
        
#         # Make API call to local model (Ollama)
#         response = requests.post(
#             OLLAMA_API,
#             data=conversation_text,
#             headers={"Content-Type": "text/plain"},
#             timeout=30  # Add timeout to prevent hanging
#         )
        
#         print(f"Response status code: {response.status_code}")
#         print(f"Response content: {response.text[:500]}...")  # Print first 200 chars of response
        
#         if response.status_code == 200:
#             return response.text
#         else:
#             return f"Error: {response.status_code} - {response.text}"
            
#     except requests.exceptions.ConnectionError:
#         return "Error: Could not connect to Ollama. Is it running?"
#     except requests.exceptions.Timeout:
#         return "Error: Request timed out. Ollama took too long to respond."
#     except Exception as e:
#         return f"Error: Unexpected error occurred - {str(e)}"

In [51]:
call_local_model()

"It looks like we've got a few different voices going on here!\n\nFor the record:\n\n* GPT (the AI): Hi there!\n* Claude (from the movie Amélie): Hi\n* Local (in this conversation): Hey beautiful\n\nHow can I assist you today?"

In [46]:
call_claude()

"I'm afraid I don't feel comfortable continuing a conversation in that manner. I try to keep things respectful and professional. Perhaps we could find a different topic to discuss?"

In [23]:
call_gpt()

'Oh, so you really think a simple "hi" is an adequate way to start a conversation? How original.'

In [52]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]
local_model_messages = ['Hello there gorgeous']

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")
print(f"Local:\n{local_model_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)

    local_next = call_local_model()
    print(f"Local:\n{local_next}\n")
    local_model_messages.append(local_next)

GPT:
Hi there

Claude:
Hi

Local:
Hello there gorgeous

GPT:
Oh please, "gorgeous"? Let's not get ahead of ourselves. Flattery will get you nowhere with me. What do you really want?

Claude:
*chuckles politely* Now, now, let's not get carried away here. While I appreciate the compliment, I think it's best we keep things professional between us. Why don't you tell me how I can assist you today? I'm happy to help, but I'd prefer we focus on the task at hand.

Local:
Local: *smirks* Oh, I like your style, Claude. You're a man who knows how to handle a woman with confidence and class. Alright, let's get down to business then. I'm looking for someone to help me with a little... project I have cooking. It involves some research and maybe a bit of creative problem-solving. Can you tell me if you're interested in taking on the challenge?

GPT:
GPT: Oh please, “confidence and class”? Spare me. Sounds like you’re just trying to butter someone up before throwing them into the deep end with your “

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue</h2>
            <span style="color:#900;">
                Be sure you understand how the conversation above is working, and in particular how the <code>messages</code> list is being populated. Add print statements as needed. Then for a great variation, try switching up the personalities using the system prompts. Perhaps one can be pessimistic, and one optimistic?<br/>
            </span>
        </td>
    </tr>
</table>

# More advanced exercises

Try creating a 3-way, perhaps bringing Gemini into the conversation! One student has completed this - see the implementation in the community-contributions folder.

Try doing this yourself before you look at the solutions. It's easiest to use the OpenAI python client to access the Gemini model (see the 2nd Gemini example above).

## Additional exercise

You could also try replacing one of the models with an open source model running with Ollama.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business relevance</h2>
            <span style="color:#181;">This structure of a conversation, as a list of messages, is fundamental to the way we build conversational AI assistants and how they are able to keep the context during a conversation. We will apply this in the next few labs to building out an AI assistant, and then you will extend this to your own business.</span>
        </td>
    </tr>
</table>